In [ ]:
import pandas as pd
import folium as fl
from folium.plugins import FastMarkerCluster,MarkerCluster,MiniMap
import branca.colormap as cm
import geopandas as gpd

# Pits and Ponds
1.  Pits
    a.  Read data and clean - returns a dataframe
    b.  aggregate, sum and sort - return a list of top n values
    c.  create map
    d.  add markers for each element of list in 2 above
    3.  create layesrs


In [ ]:
# Read data and cleans - pit data
def get_pits_df():
    
    df1 = pd.read_csv('/Users/marilynwaldman/Downloads/CA_WastewaterPits_Fractracker/CA_WastewaterPits_FracTracker.csv')
    df1['Pit Count'].replace({"Unknown":1},inplace=True)
    df1.head()
    
    df2 = pd.read_csv('/Users/marilynwaldman/Downloads/CA_WastewaterPits_Fractracker/CA_WastewaterPits_Reviseddataset_FracTracker.csv')
    #replace Pit Count "Unknown" to 1
    # drop Operator.1 from df2
    df2.drop('Operator.1',axis=1,inplace=True)
    print(df2.columns)
    
    frames = [df1, df2]
    df = pd.concat(frames)
    #replace Pit Count "Unknown" to 1
    df['Pit Count'].replace({"Unknown":1},inplace=True)
    #replace newlines with space
    df['Operator'].replace(r'\n',' ', regex=True,inplace=True) 
    #Compute number of sites
    df['Sites'] = df['Pit Count'].astype(int)
    return df

In [ ]:
df_pits = get_pits_df()
df_pits.head()

In [ ]:
#aggregate pit df and return a list of top n Operators
def get_pits_top_n(df, n):
    
    pit_hist = df.groupby('Operator').sum().sort_values(by='Sites',ascending=False)
    pitlist = pit_hist['Sites'].index.values.tolist()
    
    return pitlist[0:n]
  
    

In [ ]:
pit_list = get_pits_top_n(df_pits, 6)
pit_list

In [ ]:
def create_map():
    calmap = fl.Map(location=[34.8721, -122.2578],
                zoom_start=6,tiles=None)
    fl.TileLayer('Stamen Toner',name='BackGround',control=False).add_to(calmap)
    
    return calmap

In [ ]:
pit_map = create_map()
pit_map

In [ ]:
def add_counties(calmap):
    ca_counties_gdf = gpd.read_file("../data/california_counties/CaliforniaCounties.shp")
    fl.GeoJson(ca_counties_gdf, name='California Counties', control=False,
               style_function = lambda x: {"weight":0.5, 
                            'color':'grey',
                            'fillColor':'transparent',
#                            'fillColor':colormap_county(x['properties']['Active']), 
                            'fillOpacity':0.5
               }
           
              ).add_to(calmap)
    return calmap


In [ ]:
pit_map = add_counties(pit_map)
pit_map

In [ ]:
# getgdf (df, col_name, col_value)
# helper function

def getgdf(df, column_name, column_value):
    # input a dataframe, a column and column value
    # returns -  geopandas dataframe with lat/lon
    odf  =  df[df[column_name] == column_value]
    odfgpd = gpd.GeoDataFrame(odf,geometry=gpd.points_from_xy(odf.Longitude, odf.Latitude))
    return(odfgpd)

In [ ]:
def get_markers(mlist, df, calmap, zoom_max):
    
    for operator in mlist:
        gdfpit = getgdf(df, "Operator", operator)
        gdfpit.set_crs('EPSG:4269',inplace=True)
        marker_name = operator 
        opmarker = MarkerCluster(name=marker_name,overlay=True,control=True,disableClusteringAtZoom=zoom_max,
                              spiderfyOnMaxZoom=zoom_max)

        # Note that CircleMarkers are saved to the MarkerCluster layer, not the map
        gdfpit.apply(lambda row:
                        fl.CircleMarker(name='Pit Markers', control=True, zindexoffset=1000,
                                  location=[row['geometry'].y, row['geometry'].x],
                                  radius=5,
                                  color='orange',
                                  fill=True,
                                  fill_color='yellow',
                                  popup=row['Operator'],
                            
               highlight_function = lambda x: {'fillColor': '#000000', 
                                'color':'#000000', 
                                'fillOpacity': 0.25, 
                                'weight': 0.1
               },
               
               tooltip = "Operator: %s<br>Pits: %s" % (row['Operator'], row['Sites'])             

                                 ).add_to(opmarker), 
                         axis=1)


               
        #Add the Markeluster layer to the map
        opmarker.add_to(calmap)
        
    return(calmap)
    

In [ ]:
pit_map = get_markers(pit_list, df_pits, pit_map, zoom_max = 15)
pit_map

In [ ]:
# Finally add control layer and display map
fl.LayerControl(collapsed=False).add_to(pit_map)
pit_map
